In [47]:
!pip install cohens_d

  Preparing metadata (setup.py) ... done
  Created wheel for cohens_d: filename=cohens_d-0.1.5-py3-none-any.whl size=2222 sha256=b9269f5467e215a2e57d53a1980191070b9bc28a4823675c6693299f991f53d9
  Stored in directory: /Users/oskarroeske/Library/Caches/pip/wheels/6e/e7/62/4c967e0bf27cccb1265b7b34163dfffebfcfde6d57b3404cb2
Successfully built cohens_d


In [4]:
import pandas as pd
import numpy as np
import ast
import json
from collections import Counter, defaultdict

In [5]:
df_arguments_sentiment = pd.read_csv("result_dataframe_v2.csv")
df_paragraphs_arguments = pd.read_csv("report_arguments.csv")
df_reports = pd.read_csv("preprocessed_reports.csv")

In [6]:
df_arguments_sentiment = df_arguments_sentiment.drop(columns="Unnamed: 0")
df_paragraphs_arguments = df_paragraphs_arguments.drop(columns=["Unnamed: 0","Unnamed: 0.1","paragraph"])
df_reports = df_reports.drop(columns=["Unnamed: 0","paragraphs"])

In [7]:
list_of_arguments = [
    "Growth",
    "Price/Earnings Ratio",
    "Earnings per Share",
    "Cash Flow",
    "Revenue",
    "Return On Equity",
    "Margins",
    "Cost Management",
    "Dividend Policy",
    "Investments",
    "Balance Sheet",
    "Long-term Growth",
    "Mergers and Acquisition",
    "Refranchising",
    "Sustainability",
    "Employees",
    "Research and Development",
    "Marketing",
    "Shares Repurchase",
    "Processes",
    "Leadership",
    "Innovation",
    "Product Characteristics",
    "Pricing Strategy",
    "Production",
    "Technology Trends",
    "Market Share",
    "Market Conditions",
    "Market Expansion",
    "Competition",
    "Global Presence",
    "Industry Outlook",
    "Regulations",
    "Partnerships and Collaborations",
    "Supply Chain",
    "Economic Conditions",
    "Demand",
    "Customers"
]


In [8]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Cond..."
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership..."
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Technology Trends': 'positive', 'Growth': 'p..."
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Innovation': 'positive', 'Market Share': 'po..."
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Revenue': 'negative', ..."
...,...,...,...,...
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Revenue': 'negative', 'Market Share': 'posit..."
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi..."
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,"{'Market Share': 'positive', 'Volume Trends': ..."
12571,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,"{'Growth': 'positive', 'Revenue': 'positive'}"


# 1. Calculate occurences of positive/negative words

In [9]:
# Turn string dictionary back to actual dictionary
df_paragraphs_arguments["provided_arguments"] = df_paragraphs_arguments["provided_arguments"].apply(ast.literal_eval)

In [10]:
from collections import Counter

# Step 2: Count occurrences of 'positive', 'negative', 'neutral' in each row
def count_sentiments(arguments):
    counts = Counter(arguments.values())  # Count occurrences of each sentiment
    return {'positive_arguments': counts.get('positive', 0),
            'neutral_arguments': counts.get('neutral', 0),
            'negative_arguments': counts.get('negative', 0)}

df_counts_paragraph = df_paragraphs_arguments['provided_arguments'].apply(count_sentiments)
df_counts_paragraph = pd.DataFrame(df_counts_paragraph.tolist())  # Expand the dictionary into separate columns

df_paragraphs_arguments = pd.concat([df_paragraphs_arguments, df_counts_paragraph], axis=1)

In [11]:
df_paragraphs_arguments.head(10)

,filename,document_id,paragraph_id,provided_arguments,positive_arguments,neutral_arguments,negative_arguments
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Cond...",1,0,1
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership...",2,0,0
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Technology Trends': 'positive', 'Growth': 'p...",2,0,0
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Innovation': 'positive', 'Market Share': 'po...",2,0,1
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Revenue': 'negative', ...",0,0,3
5,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,1,"{'Growth': 'positive', 'Market Share': 'positi...",3,0,0
6,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,2,"{'Growth': 'positive', 'Revenue': 'positive', ...",3,0,2
7,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,3,"{'Economic Conditions': 'negative', 'Competiti...",0,0,5
8,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,1,"{'Margins': 'negative', 'Competition': 'negati...",0,0,3
9,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,2,"{'Growth': 'positive', 'Market Conditions': 'n...",1,0,3


In [12]:
df_counts_reports = df_paragraphs_arguments.groupby('filename',as_index=False)[['positive_arguments','negative_arguments','neutral_arguments']].sum()

In [13]:
df_reports = df_reports.merge(df_counts_reports,on="filename",how='inner',)
#df_merged = df_saved_reports.merge(df_updated_reports, on='filename', how='left', suffixes=('', '_new'))

In [14]:
df_reports["tp_reached"] = df_reports[["tp reached after 3 months","tp reached after 6 months","tp reached after 9 months","tp reached after 12 months"]].any(axis=1)

In [15]:
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,min price after 12 months,adjusted_target_price,tp reached after 3 months,tp reached after 6 months,tp reached after 9 months,tp reached after 12 months,positive_arguments,negative_arguments,neutral_arguments,tp_reached
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,155.27,150.00,False,False,True,True,16,2,0,True
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,302.55,300.00,True,False,True,True,17,5,0,True
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,48.18,39.00,True,True,True,True,25,1,0,True
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,2144.72,1950.00,False,True,True,True,11,5,0,True
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,18.12,50.00,False,False,False,False,17,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,54.77,NaN,False,False,False,False,17,3,0,False
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,250.93,270.00,True,True,True,True,18,2,0,True
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,40.68,55.00,True,True,True,True,24,7,0,True
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,37.47,44.00,False,False,False,True,40,14,3,True


In [16]:
df_reports.describe()

,document_id,target_price,start price,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months,adjusted_target_price,positive_arguments,negative_arguments,neutral_arguments
count,916.000000,705.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.000000,916.00000,916.000000,705.000000,916.000000,916.000000,916.000000
mean,458.893013,239.434221,142.092566,142.414050,159.867402,126.860568,161.215819,131.606976,163.261965,135.089378,164.46107,137.129541,163.623021,19.688865,7.514192,1.519651
std,264.984263,457.773704,268.928808,271.943514,306.917958,240.531514,308.106246,251.197388,310.245496,255.830534,314.73155,259.486271,299.851799,19.512034,8.312662,2.378098
min,1.000000,0.500000,0.400000,0.410000,0.470000,0.380000,0.490000,0.450000,0.500000,0.440000,0.53000,0.420000,0.100000,0.000000,0.000000,0.000000
25%,229.750000,55.000000,36.502500,36.257500,40.560000,31.745000,41.290000,33.057500,41.492500,33.640000,41.29000,34.090000,45.000000,9.000000,2.000000,0.000000
50%,458.500000,110.000000,71.020000,71.160000,79.890000,62.810000,80.800000,66.805000,82.895000,69.945000,85.50000,71.170000,88.000000,14.000000,5.000000,1.000000
75%,688.250000,240.000000,164.250000,163.630000,178.760000,141.112500,179.557500,144.832500,180.950000,151.560000,180.15000,154.062500,200.000000,23.000000,9.000000,2.000000
max,917.000000,4400.000000,2464.930000,2618.970000,2703.260000,2172.250000,2703.260000,2172.190000,2703.260000,2263.790000,2703.26000,2144.720000,4400.000000,149.000000,90.000000,32.000000


In [17]:
df_reports["total_arguments"] = df_reports[["positive_arguments","negative_arguments","neutral_arguments"]].sum(axis=1)

In [18]:
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,adjusted_target_price,tp reached after 3 months,tp reached after 6 months,tp reached after 9 months,tp reached after 12 months,positive_arguments,negative_arguments,neutral_arguments,tp_reached,total_arguments
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,150.00,False,False,True,True,16,2,0,True,18
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,300.00,True,False,True,True,17,5,0,True,22
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,39.00,True,True,True,True,25,1,0,True,26
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,1950.00,False,True,True,True,11,5,0,True,16
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,50.00,False,False,False,False,17,2,1,False,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,NaN,False,False,False,False,17,3,0,False,20
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,270.00,True,True,True,True,18,2,0,True,20
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,55.00,True,True,True,True,24,7,0,True,31
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,44.00,False,False,False,True,40,14,3,True,57


# Argument Analysis

In [19]:
df_arguments_sentiment

,Argument,positive,neutral,negative
0,Technology Trends,272,7,14
1,Market Conditions,642,215,790
2,Partnerships and Collaborations,203,3,4
3,Growth,3409,75,447
4,Innovation,446,5,28
...,...,...,...,...
268,Investor Sentiment,3,0,0
269,Operational Risk,0,0,2
270,Consumer Engagement,2,0,0
271,Deal-Making,1,0,0


In [20]:
#Has to be done later
df_arguments_sentiment
#Plan: Assign similar categories to the same broad category

# Create map to merge very close arguments
mapping = {
    "Shares Repurchase": ["Buybacks", "Share Buyback", "Share rebuy"],
    "Partnerships and Collaboration":["Partnerships","Collaboration"],
    "Pricing Strategy":["Pricing"],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Revenue":["Ad Revenue"],
    "Regulations":["Regulatory Approval"],
    "Cash Flow":["Free Cash Flow"],
    "Marketing":["Brand Strength"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_arguments_sentiment["Argument"] = df_arguments_sentiment["Argument"].replace(reverse_mapping)

# Group By the broad category to assign the numbers
df_aggregated = df_arguments_sentiment.groupby("Argument").sum(numeric_only=True).reset_index()

# Display the result
df_aggregated

,Argument,positive,neutral,negative
0,Accountability,1,0,2
1,Accounts,1,0,0
2,Accounts Payable,0,0,1
3,Advertiser Engagement,5,0,0
4,Advertising,9,1,1
...,...,...,...,...
264,Volume Trends,2,1,0
265,Workflows,1,0,0
266,Working Capital,2,0,0
267,Yield,1,0,1


# Argument Analysis success of Reports

In [21]:
df_success_evaluation_reports = pd.merge(df_paragraphs_arguments,df_reports[["filename","tp_reached","rating","industry"]],on="filename",how='left')

## Successful reports

In [22]:
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

In [23]:
#Should I reassign the arguments here??

# Get dataframe for successful recommendations
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_success = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_success_reports["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts_success[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
df_sentiment_success = pd.DataFrame(sentiment_counts_success).T
df_sentiment_success.columns = ["positive", "neutral", "negative"]
df_sentiment_success.index.name = "Argument"

# Display the overview DataFrame
df_sentiment_success.reset_index(inplace=True)
df_sentiment_success["total"] = df_sentiment_success[["positive","negative","neutral"]].sum(axis=1)
df_sentiment_success

,Argument,positive,neutral,negative,total
0,Growth,1580,37,199,1816
1,Market Share,166,1,31,198
2,Revenue,1562,55,315,1932
3,Margins,403,29,258,690
4,Earnings per Share,511,46,160,717
...,...,...,...,...,...
218,Pricing Power,2,0,1,3
219,Brand Loyalty,1,0,0,1
220,Currency Impact,0,0,1,1
221,Visibility,1,0,0,1


In [24]:
## Mapping before ratios are getting created
mapping = {
    "Shares Repurchase": ["Buybacks", "Share Buyback", "Share rebuy"],
    "Partnerships and Collaboration":["Partnerships","Collaboration"],
    "Pricing Strategy":["Pricing"],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Revenue":["Ad Revenue"],
    "Regulations":["Regulatory Approval"],
    "Cash Flow":["Free Cash Flow"],
    "Marketing":["Brand Strength"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_sentiment_success["Argument"] = df_sentiment_success["Argument"].replace(reverse_mapping)

# Group By the broad category to assign the numbers
df_sentiment_success = df_sentiment_success.groupby("Argument").sum(numeric_only=True).reset_index()
df_sentiment_success

,Argument,positive,neutral,negative,total
0,Accountability,0,0,1,1
1,Accounts,1,0,0,1
2,Accounts Payable,0,0,1,1
3,Advertiser Engagement,1,0,0,1
4,Advertising,4,0,0,4
...,...,...,...,...,...
214,Visibility,1,0,0,1
215,Volatility,1,1,6,8
216,Volume,0,0,2,2
217,Yield,0,0,1,1


In [25]:
# Create Ratio fields
df_sentiment_success["argument_importance"] = round((df_sentiment_success["total"] / df_sentiment_success["total"].sum()),2)
df_sentiment_success["positive_ratio"] = round((df_sentiment_success["positive"] / df_sentiment_success["total"]),2)
df_sentiment_success["negative_ratio"] = round((df_sentiment_success["negative"] / df_sentiment_success["total"]),2)
df_sentiment_success["neutral_ratio"] = round((df_sentiment_success["neutral"] / df_sentiment_success["total"]),2)
#df_sentiment_success["positive"] = df_sentiment_success["positive"].apply(lambda x: x.int())
df_sentiment_success = df_sentiment_success.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_success["position"] = range(1,len(df_sentiment_success)+1)
df_sentiment_success.head(30)

,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,position
100,Growth,1580,37,199,1816,0.15,0.87,0.11,0.02,1
186,Revenue,1563,55,315,1933,0.16,0.81,0.16,0.03,2
71,Earnings per Share,511,46,160,717,0.06,0.71,0.22,0.06,3
133,Margins,403,29,258,690,0.06,0.58,0.37,0.04,4
134,Market Conditions,313,104,358,775,0.06,0.40,0.46,0.13,5
23,Cash Flow,239,21,84,344,0.03,0.69,0.24,0.06,6
107,Innovation,234,1,19,254,0.02,0.92,0.07,0.00,7
114,Investments,229,10,57,296,0.02,0.77,0.19,0.03,8
135,Market Expansion,228,2,5,235,0.02,0.97,0.02,0.01,9
38,Cost Management,191,13,82,286,0.02,0.67,0.29,0.05,10


# Argument Analysis not Successful Reports

In [26]:
df_unsuccess_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==False]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_unsuccess = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_unsuccess_reports["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts_unsuccess[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
df_sentiment_unsuccess = pd.DataFrame(sentiment_counts_unsuccess).T
df_sentiment_unsuccess.columns = ["positive", "neutral", "negative"]
df_sentiment_unsuccess.index.name = "Argument"

# Display the overview DataFrame
df_sentiment_unsuccess.reset_index(inplace=True)
df_sentiment_unsuccess["total"] = df_sentiment_unsuccess[["positive","negative","neutral"]].sum(axis=1)
df_sentiment_unsuccess

,Argument,positive,neutral,negative,total
0,Technology Trends,133,2,5,140
1,Market Conditions,329,111,432,872
2,Partnerships and Collaborations,108,3,4,115
3,Growth,1829,38,248,2115
4,Innovation,212,4,9,225
...,...,...,...,...,...
210,Investor Sentiment,3,0,0,3
211,Loan Loss Reserve,0,0,1,1
212,Operational Risk,0,0,2,2
213,Consumer Engagement,2,0,0,2


In [27]:
## Mapping before ratios are getting created
mapping = {
    "Shares Repurchase": ["Buybacks", "Share Buyback", "Share rebuy"],
    "Partnerships and Collaboration":["Partnerships","Collaboration"],
    "Pricing Strategy":["Pricing"],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Revenue":["Ad Revenue"],
    "Regulations":["Regulatory Approval"],
    "Cash Flow":["Free Cash Flow"],
    "Marketing":["Brand Strength"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_sentiment_unsuccess["Argument"] = df_sentiment_unsuccess["Argument"].replace(reverse_mapping)

# Group By the broad category to assign the numbers
df_sentiment_unsuccess = df_sentiment_unsuccess.groupby("Argument").sum(numeric_only=True).reset_index()
df_sentiment_unsuccess


,Argument,positive,neutral,negative,total
0,Accountability,1,0,1,2
1,Advertiser Engagement,4,0,0,4
2,Advertising,5,1,1,7
3,Asset Management,1,0,0,1
4,Asset Quality,2,0,1,3
...,...,...,...,...,...
206,Volume,1,0,1,2
207,Volume Trends,2,1,0,3
208,Workflows,1,0,0,1
209,Working Capital,2,0,0,2


In [28]:
# Create Ratio fields
df_sentiment_unsuccess["argument_importance"] = round((df_sentiment_unsuccess["total"] / df_sentiment_unsuccess["total"].sum()),2)
df_sentiment_unsuccess["positive_ratio"] = round((df_sentiment_unsuccess["positive"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess["negative_ratio"] = round((df_sentiment_unsuccess["negative"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess["neutral_ratio"] = round((df_sentiment_unsuccess["neutral"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess = df_sentiment_unsuccess.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_unsuccess["position"] = range(1,len(df_sentiment_unsuccess)+1)

df_sentiment_unsuccess.head(30)

,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,position
178,Revenue,1829,65,423,2317,0.16,0.79,0.18,0.03,1
90,Growth,1829,38,248,2115,0.15,0.86,0.12,0.02,2
63,Earnings per Share,659,66,216,941,0.07,0.70,0.23,0.07,3
121,Margins,482,25,300,807,0.06,0.60,0.37,0.03,4
17,Cash Flow,378,31,107,516,0.04,0.73,0.21,0.06,5
122,Market Conditions,329,111,432,872,0.06,0.38,0.50,0.13,6
103,Investments,281,12,70,363,0.03,0.77,0.19,0.03,7
29,Cost Management,258,10,103,371,0.03,0.70,0.28,0.03,8
97,Innovation,212,4,9,225,0.02,0.94,0.04,0.02,9
123,Market Expansion,189,1,2,192,0.01,0.98,0.01,0.01,10


https://stackoverflow.com/questions/52783391/how-to-merge-dictionaries-of-a-pandas-dataframe-when-grouping-by-rows

# DIfference between succefull / not successful arguments

In [29]:
# Merge the two DataFrames on 'Argument' to align rows
merged_df = pd.merge(df_sentiment_success, df_sentiment_unsuccess, on='Argument', suffixes=('_df1', '_df2'))
merged_df["average_total"] = (merged_df["total_df1"] + merged_df["total_df2"])/2

In [39]:
merged_df.head(30)

,Argument,positive_df1,neutral_df1,negative_df1,total_df1,argument_importance_df1,positive_ratio_df1,negative_ratio_df1,neutral_ratio_df1,position_df1,...,negative_df2,total_df2,argument_importance_df2,positive_ratio_df2,negative_ratio_df2,neutral_ratio_df2,position_df2,average_total,relative_usage_df1,relative_usage_df2
0,Growth,1580,37,199,1816,0.15,0.87,0.11,0.02,1,...,248,2115,0.15,0.86,0.12,0.02,2,1965.5,0.151245,0.149735
1,Revenue,1563,55,315,1933,0.16,0.81,0.16,0.03,2,...,423,2317,0.16,0.79,0.18,0.03,1,2125.0,0.160989,0.164035
2,Earnings per Share,511,46,160,717,0.06,0.71,0.22,0.06,3,...,216,941,0.07,0.70,0.23,0.07,3,829.0,0.059715,0.066619
3,Margins,403,29,258,690,0.06,0.58,0.37,0.04,4,...,300,807,0.06,0.60,0.37,0.03,4,748.5,0.057466,0.057133
4,Market Conditions,313,104,358,775,0.06,0.40,0.46,0.13,5,...,432,872,0.06,0.38,0.50,0.13,6,823.5,0.064546,0.061735
5,Cash Flow,239,21,84,344,0.03,0.69,0.24,0.06,6,...,107,516,0.04,0.73,0.21,0.06,5,430.0,0.028650,0.036531
6,Innovation,234,1,19,254,0.02,0.92,0.07,0.00,7,...,9,225,0.02,0.94,0.04,0.02,9,239.5,0.021154,0.015929
7,Investments,229,10,57,296,0.02,0.77,0.19,0.03,8,...,70,363,0.03,0.77,0.19,0.03,7,329.5,0.024652,0.025699
8,Market Expansion,228,2,5,235,0.02,0.97,0.02,0.01,9,...,2,192,0.01,0.98,0.01,0.01,10,213.5,0.019572,0.013593
9,Cost Management,191,13,82,286,0.02,0.67,0.29,0.05,10,...,103,371,0.03,0.70,0.28,0.03,8,328.5,0.023819,0.026265


In [31]:
# Calculate differences for all numeric columns (excluding 'Argument')
difference_df = merged_df[['Argument',"average_total"]].copy()  # Start with the 'Argument' column
columns_to_compare = [col for col in df_sentiment_success.columns if col != 'Argument' and col != 'average_total']  # Columns to compute differences for

for col in columns_to_compare:
    difference_df[col + '_difference'] = merged_df[col + '_df1'] - merged_df[col + '_df2']



In [32]:
difference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Argument                        161 non-null    object 
 1   average_total                   161 non-null    float64
 2   positive_difference             161 non-null    int64  
 3   neutral_difference              161 non-null    int64  
 4   negative_difference             161 non-null    int64  
 5   total_difference                161 non-null    int64  
 6   argument_importance_difference  161 non-null    float64
 7   positive_ratio_difference       161 non-null    float64
 8   negative_ratio_difference       161 non-null    float64
 9   neutral_ratio_difference        161 non-null    float64
 10  position_difference             161 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 14.0+ KB


In [33]:
difference_df.to_csv("difference_testing.csv")

In [40]:
difference_df

,Argument,average_total,positive_difference,neutral_difference,negative_difference,total_difference,argument_importance_difference,positive_ratio_difference,negative_ratio_difference,neutral_ratio_difference,position_difference
0,Growth,1965.5,-249,-1,-49,-299,0.00,0.01,-0.01,0.00,-1
1,Revenue,2125.0,-266,-10,-108,-384,0.00,0.02,-0.02,0.00,1
2,Earnings per Share,829.0,-148,-20,-56,-224,-0.01,0.01,-0.01,-0.01,0
3,Margins,748.5,-79,4,-42,-117,0.00,-0.02,0.00,0.01,0
4,Market Conditions,823.5,-16,-7,-74,-97,0.00,0.02,-0.04,0.00,-1
...,...,...,...,...,...,...,...,...,...,...,...
156,Yield,1.0,-1,0,1,0,0.00,-1.00,1.00,0.00,23
157,Asset Sensitivity,1.0,0,0,0,0,0.00,0.00,0.00,0.00,5
158,Drilling Results,1.0,-1,1,0,0,0.00,-1.00,0.00,1.00,57
159,EBITDA,2.5,-1,0,-2,-3,0.00,-0.25,-0.50,0.75,74


# Statistical Significance
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [49]:
import numpy as np

def cohens_d(group1, group2):
    # Means of the groups
    mean1, mean2 = np.mean(group1), np.mean(group2)
    
    # Standard deviations of the groups
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # Sizes of the groups
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Compute Cohen's d
    return (mean1 - mean2) / pooled_std

In [55]:
import numpy as np

# Extract total success and unsuccess counts
success_counts = merged_df["total_df1"].tolist()
unsuccess_counts = merged_df["total_df2"].tolist()

cohens_d_values = []

for success, unsuccess in zip(success_counts, unsuccess_counts):
    # Means are the counts themselves
    mean_success = success
    mean_unsuccess = unsuccess
    
    # Approximate standard deviations (for count data, use sqrt(count))
    std_success = np.sqrt(success)  # Estimated standard deviation for success
    std_unsuccess = np.sqrt(unsuccess)  # Estimated standard deviation for unsuccess
    
    # Compute pooled standard deviation
    pooled_std = np.sqrt(((std_success**2 + std_unsuccess**2) / 2))
    
    # Compute Cohen's d
    d = (mean_success - mean_unsuccess) / pooled_std
    cohens_d_values.append(d)

# Add Cohen's d values to the DataFrame
merged_df["cohen_d"] = cohens_d_values

# Display results
print(merged_df[["Argument", "cohen_d"]])

               Argument   cohen_d
0                Growth -6.744266
1               Revenue -8.330130
2    Earnings per Share -7.779842
3               Margins -4.276515
4     Market Conditions -3.380181
..                  ...       ...
156               Yield  0.000000
157   Asset Sensitivity  0.000000
158    Drilling Results  0.000000
159              EBITDA -1.897367
160            Reserves -2.672612

[161 rows x 2 columns]


In [56]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
import pingouin as pg


# Step 1: Chi-square test for overall usage
merged_df["relative_usage_df1"] = merged_df["total_df1"] / merged_df["total_df1"].sum()
merged_df["relative_usage_df2"] = merged_df["total_df2"] / merged_df["total_df2"].sum()

# Create lists to store test results
statistical_significance_values = []

# Loop through each argument
for _, row in merged_df.iterrows():
    argument = row["Argument"]

    successful_count = row["total_df1"]
    not_successful_count = row["total_df2"]
    other_successful_count = sum(merged_df["total_df1"]) - successful_count
    other_not_successful_count = sum(merged_df["total_df2"]) - not_successful_count

    contingency_table = [[successful_count, other_successful_count],
                     [not_successful_count, other_not_successful_count]]

    chi2_stat, p_total, _, _ = stats.chi2_contingency(contingency_table)

    print(contingency_table)

    # Step 2: Proportion z-test for relative usage
    count = [row["relative_usage_df1"] * sum(merged_df["total_df1"]),
             row["relative_usage_df2"] * sum(merged_df["total_df2"])]
    nobs = [sum(merged_df["total_df1"]), sum(merged_df["total_df2"])]
    z_stat, p_relative = proportions_ztest(count, nobs)

    # Compute Cohen's d for each argument

    # Store results
    statistical_significance_values.append({
        "Argument": argument,
        "p_total": p_total,
        "p_relative": p_relative,
        "z_stat":z_stat,
        "success_count":row["total_df1"],
        "unsuccess_count":row["total_df2"],
        "cohen_d":row["cohen_d"]
    })

# Create a DataFrame for results
df_statistical_significance = pd.DataFrame(statistical_significance_values)

# Display the results
df_statistical_significance.head(30)

[[1816, 10191], [2115, 12010]]
[[1933, 10074], [2317, 11808]]
[[717, 11290], [941, 13184]]
[[690, 11317], [807, 13318]]
[[775, 11232], [872, 13253]]
[[344, 11663], [516, 13609]]
[[254, 11753], [225, 13900]]
[[296, 11711], [363, 13762]]
[[235, 11772], [192, 13933]]
[[286, 11721], [371, 13754]]
[[198, 11809], [206, 13919]]
[[165, 11842], [175, 13950]]
[[153, 11854], [140, 13985]]
[[165, 11842], [210, 13915]]
[[146, 11861], [166, 13959]]
[[95, 11912], [115, 14010]]
[[100, 11907], [157, 13968]]
[[155, 11852], [230, 13895]]
[[122, 11885], [148, 13977]]
[[82, 11925], [111, 14014]]
[[67, 11940], [118, 14007]]
[[97, 11910], [112, 14013]]
[[61, 11946], [50, 14075]]
[[58, 11949], [59, 14066]]
[[124, 11883], [105, 14020]]
[[79, 11928], [78, 14047]]
[[74, 11933], [72, 14053]]
[[65, 11942], [88, 14037]]
[[55, 11952], [71, 14054]]
[[279, 11728], [295, 13830]]
[[51, 11956], [46, 14079]]
[[57, 11950], [54, 14071]]
[[62, 11945], [86, 14039]]
[[54, 11953], [89, 14036]]
[[56, 11951], [54, 14071]]
[[39, 1

,Argument,p_total,p_relative,z_stat,success_count,unsuccess_count,cohen_d
0,Growth,0.746655,0.733545,0.340414,1816,2115,-6.744266
1,Revenue,0.516902,0.506085,-0.664946,1933,2317,-8.330130
2,Earnings per Share,0.024051,0.022503,-2.281775,717,941,-7.779842
3,Margins,0.929092,0.907895,0.115694,690,807,-4.276515
4,Market Conditions,0.364724,0.351368,0.931940,775,872,-3.380181
5,Cash Flow,0.000425,0.000372,-3.558862,344,516,-8.294577
6,Innovation,0.001990,0.001701,3.138033,254,225,1.873895
7,Investments,0.618272,0.590655,-0.537887,296,363,-3.691026
8,Market Expansion,0.000177,0.000145,3.799298,235,192,2.942859
9,Cost Management,0.222822,0.208138,-1.258702,286,371,-4.689767


## Create new columns
1. Add TP_Reached_as one column
2. Add total_as_arguments one column

# 2. Replace similar arguments with broader category

# 3. Calculate number of positive / negative / neutral Arguments

# Last Step: Merge Dictionaries

# Report Analysis

In [484]:
#1. Step -> create additional columns that contain the arguments/sentiments as a list, this way the argument can be multiple times per document (not possibel in dictionary)
df_paragraphs_arguments['argument_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.keys()))
df_paragraphs_arguments['sentiment_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.values()))

In [485]:
## Mapping before ratios are getting created
mapping = {
    "Shares Repurchase": ["Buybacks", "Share Buyback", "Share rebuy"],
    "Partnerships and Collaboration":["Partnerships","Collaboration"],
    "Pricing Strategy":["Pricing"],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Revenue":["Ad Revenue"],
    "Regulations":["Regulatory Approval"],
    "Cash Flow":["Free Cash Flow"],
    "Marketing":["Brand Strength"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_paragraphs_arguments["argument_list"] = df_paragraphs_arguments["argument_list"].replace(reverse_mapping)

In [486]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments,positive_arguments,neutral_arguments,negative_arguments,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,1,"{'Technology Trends': 'positive', 'Market Conditions': 'negative'}",1,0,1,"[Technology Trends, Market Conditions]","[positive, negative]"
1,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,2,"{'Technology Trends': 'positive', 'Partnerships and Collaborations': 'positive'}",2,0,0,"[Technology Trends, Partnerships and Collaborations]","[positive, positive]"
2,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,3,"{'Technology Trends': 'positive', 'Growth': 'positive'}",2,0,0,"[Technology Trends, Growth]","[positive, positive]"
3,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,4,"{'Innovation': 'positive', 'Market Share': 'positive', 'Competition': 'negative'}",2,0,1,"[Innovation, Market Share, Competition]","[positive, positive, negative]"
4,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,5,"{'Growth': 'negative', 'Revenue': 'negative', 'Market Conditions': 'negative'}",0,0,3,"[Growth, Revenue, Market Conditions]","[negative, negative, negative]"
...,...,...,...,...,...,...,...,...,...
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_China_October_Data_Confirm_Industry.pdf,203,1,"{'Revenue': 'negative', 'Market Share': 'positive', 'Supply Chain': 'negative'}",1,0,2,"[Revenue, Market Share, Supply Chain]","[negative, positive, negative]"
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_China_October_Data_Confirm_Industry.pdf,203,2,"{'Market Share': 'positive', 'Growth': 'positive'}",2,0,0,"[Market Share, Growth]","[positive, positive]"
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_China_October_Data_Confirm_Industry.pdf,203,3,"{'Market Share': 'positive', 'Volume Trends': 'neutral'}",1,1,0,"[Market Share, Volume Trends]","[positive, neutral]"
12571,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_China_October_Data_Confirm_Industry.pdf,203,4,"{'Growth': 'positive', 'Revenue': 'positive'}",2,0,0,"[Growth, Revenue]","[positive, positive]"


In [487]:
#2. Step -> Merge argument_list and sentiment_list on document level 
# merge dictionaries
def merge_lists(x):
    return [item for sublist in x.dropna() for item in sublist]

df_reports_arguments = df_paragraphs_arguments.groupby(['filename', 'document_id'], as_index=False).agg({
    'argument_list': merge_lists,
    'sentiment_list': merge_lists
})

# Display the result
df_reports_arguments

,filename,document_id,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technologies-_but_Market_Penetration_R.pdf,512,"[Technology Trends, Market Conditions, Technology Trends, Partnerships and Collaborations, Technology Trends, Growth, Innovation, Market Share, Competition, Growth, Revenue, Market Conditions]","[positive, negative, positive, positive, positive, positive, positive, positive, negative, negative, negative, negative]"
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mixed_In_Among_the_Diamonds-_Tweaking.pdf,909,"[Growth, Market Share, Revenue, Growth, Revenue, Margins, Earnings per Share, Market Share, Economic Conditions, Competition, Cost Management, Market Conditions, Customers]","[positive, positive, positive, positive, positive, negative, negative, positive, negative, negative, negative, negative, negative]"
2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-_But_Not_Solid_All_Around-_Reiteratin.pdf,717,"[Margins, Competition, Growth, Growth, Market Conditions, Competition, Demand, Growth, Competition, Market Conditions]","[negative, negative, negative, positive, negative, negative, negative, negative, negative, negative]"
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Growth_Ahead-_But_Margin_Compression_Too.pdf,917,"[Margins, Revenue, Investments, Margins, Revenue, Margins, Long-term Growth, Revenue, Growth, Margins, Earnings per Share, Growth, Cash Flow, Revenue, Margins, Cost Management, Growth, Market Conditions, Margins, Earnings per Share, Revenue, Market Share, Growth, Revenue, Margins, Revenue, Revenue, Growth, Margins, Cost Management, Revenue, Margins, Revenue, Investments, Research and Development, Marketing, Cost Management, Margins, Earnings per Share, Revenue, Margins, Revenue, Investments, Margins, Margins, Margins, Investments, Market Conditions, Leadership, Complexity, Revenue, Margins, Long-term Growth, Competition, Investments, Revenue, Market Conditions, Economic Conditions, Regulations, Market Conditions, Competition, Regulations]","[negative, positive, negative, negative, negative, positive, neutral, positive, positive, negative, positive, positive, positive, positive, negative, negative, positive, negative, negative, positive, positive, positive, positive, positive, negative, positive, positive, positive, negative, negative, negative, negative, neutral, negative, negative, negative, negative, negative, negative, negative, negative, positive, negative, negative, positive, negative, negative, negative, negative, negative, positive, negative, positive, negative, positive, positive, negative, negative, negative, negative, negative, negative]"
4,20140124_Hilliard_Lyons_PG_Report_received_in_email..pdf,574,"[Revenue, Growth, Margins, Earnings per Share, Cost Management, Revenue, Earnings per Share, Earnings per Share, Dividend Policy, Growth, Growth, Market Share, Revenue, Revenue, Cost Management, Market Share, Margins, Earnings per Share, Cost Management, Balance Sheet, Cash Flow, Shares Repurchase, Dividend Policy, Growth, Cost Management, Investments, Market Conditions, Growth, Revenue, Earnings per Share, Revenue, Margins, Earnings per Share, Growth, Margins, Shares Repurchase, Earnings per Share, Dividend Policy, Earnings per Share, Market Conditions, Price/Earnings Ratio, Long-term Growth, Market Share, Earnings per Share, Cost Management, Investments, Price/Earnings Ratio, Earnings per Share, Revenue, Market Conditions, Dividend Policy, Long-term Growth, Revenue, Market Conditions, Supply Chain, Earnings per Share]","[positive, positive, negative, positive, positive, positive, positive, positive, positive, positive, positive, neutral, positive, positive, positive, negative, negative, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, positive, neutral, positive, negative, negative, positive, positive, positive, positive, positive, positive, n

In [488]:
from collections import Counter

# Function to get top 3 most frequent arguments
def get_top_arguments(arg_list, n):
    counter = Counter(arg_list)  # Count occurrences of each argument
    most_common = [item[0] for item in counter.most_common(n)]  # Extract top `n` arguments
    return most_common

# Apply the function to argument_list and create new columns
df_reports_arguments['1st_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 1)[0] if len(get_top_arguments(x, 1)) > 0 else None)
df_reports_arguments['2nd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 2)[1] if len(get_top_arguments(x, 2)) > 1 else None)
df_reports_arguments['3rd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 3)[2] if len(get_top_arguments(x, 3)) > 2 else None)

In [489]:
df_reports_arguments.value_counts("1st_most_used_argument")

1st_most_used_argument
Revenue                     354
Growth                      272
Earnings per Share           49
Cash Flow                    36
Margins                      31
Market Conditions            28
Clinical Trials              24
Risks                        13
Innovation                    9
Production                    7
Competition                   7
Investments                   6
Cost Management               6
Guidance                      5
Management                    5
Leadership                    4
Technology Trends             4
Pricing Strategy              3
Costs                         3
Risk Management               3
Dividend Policy               3
Free Cash Flow                3
Financial Condition           3
Capacity                      2
Market Share                  2
Profitability                 2
Sales                         2
Performance                   2
Debt                          2
Demand                        2
Market Expansion 

In [490]:
df_reports_arguments.value_counts("2nd_most_used_argument")

2nd_most_used_argument
Growth                255
Revenue               209
Earnings per Share     73
Market Conditions      71
Margins                66
                     ... 
Liquidity               1
Legal Risks             1
Fees                    1
Inventory               1
Acquisitions            1
Name: count, Length: 63, dtype: int64

In [491]:
df_reports_arguments.value_counts("3rd_most_used_argument")

3rd_most_used_argument
Earnings per Share        148
Market Conditions         123
Margins                   102
Growth                     95
Revenue                    76
                         ... 
Membership                  1
Operating Performance       1
Operational Efficiency      1
Expenses                    1
Working Capital             1
Name: count, Length: 84, dtype: int64

In [492]:
df_final_reports = pd.merge(df_reports,df_reports_arguments,on='filename',how='inner')

In [493]:
pd.reset_option('display.max_colwidth', None)  # Don't truncate column content
df_final_reports

,document_id_x,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,negative_arguments,neutral_arguments,tp_reached,total_arguments,document_id_y,argument_list,sentiment_list,1st_most_used_argument,2nd_most_used_argument,3rd_most_used_argument
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,2,0,True,18,1,"[Revenue, Growth, Earnings per Share, Operatin...","[positive, positive, positive, positive, posit...",Revenue,Growth,Earnings per Share
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,5,0,True,22,2,"[Revenue, Cost Management, Ad Revenue, Guidanc...","[positive, positive, positive, positive, negat...",Revenue,Growth,Market Conditions
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,1,0,True,26,3,"[Growth, Investments, Market Expansion, Innova...","[positive, positive, positive, positive, negat...",Growth,Investments,Innovation
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,5,0,True,16,4,"[Demand, Market Conditions, Long-term Growth, ...","[positive, positive, positive, negative, negat...",Market Conditions,Revenue,Profitability
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,2,1,False,20,5,"[Cash Flow, Growth, Investments, Earnings Perf...","[positive, positive, positive, neutral, positi...",Cash Flow,Growth,Earnings per Share
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,3,0,False,20,913,"[Growth, Earnings per Share, Dividend Policy, ...","[positive, positive, positive, positive, posit...",Growth,Revenue,Earnings per Share
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,2,0,True,20,914,"[Growth, Revenue, Market Share, Profitability,...","[positive, positive, positive, positive, posit...",Revenue,Growth,Market Share
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,7,0,True,31,915,"[Guidance, Revenue, Costs, Revenue, Earnings O...","[positive, positive, positive, positive, posit...",Revenue,Costs,Earnings Performance
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,14,3,True,57,916,"[Earnings per Share, Revenue, Valuation, Marke...","[positive, positive, negative, negative, posit...",Earnings per Share,Revenue,Valuation


In [494]:
df_final_reports.to_csv("final_reports_data.csv")